In [28]:
from transformers import BertForSequenceClassification
from transformers import RobertaForSequenceClassification
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, ElectraTokenizer

import pandas as pd
from torch.utils.data import Dataset
import torch
import re
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.max_columns', None)


In [29]:
#Choose between bert, roberta, or electra
model_type = "roberta"

In [30]:
if model_type == "bert":
    model = BertForSequenceClassification.from_pretrained("../../Models/Second_BERT/")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
elif model_type == "roberta":
    model = RobertaForSequenceClassification.from_pretrained("../../Models/RoBERTa/")
    tokenizer = AutoTokenizer.from_pretrained("roberta-base")
elif model_type == "electra":
    model = ElectraForSequenceClassification.from_pretrained("../../Models/ELECTRA/")
    tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")
    

In [31]:
#file_path = '../../Data/Policy_docs/USA/Fairness, Ethics, Accountability, and Transparency/Fairness, Ethics, Accountability, and Transparency.txt'
file_path = '../../Data/Policy_docs/USA/AI and Society/AI and Society.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

policy_text = text
policy_text = policy_text.replace('\n', '')

policy_text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', policy_text)

policy_sentences = policy_text.split('.')
policy_sentences = [s.strip() for s in policy_sentences if s.strip()]
policy_sentences = [s for s in policy_sentences if '@' not in s]
policy_sentences = [s for s in policy_sentences if len(s) >= 10]
policy_sentences = [s for s in policy_sentences if 'et al' not in s]
# Add test sentences for comparison (generated by ChatGPT) Prompt: I am trying to create an NLP model that tries to classify which types of biases an AI policy document is trying to adhere to. The five types are: gender, sexual, religion, race, and disability bias. For testing, give me an example sentence for each bias jus described that could be found in AI policy documents  that tries to prevent that type of bias.
policy_sentences.append("The AI system shall treat all genders with equal respect, ensuring that its algorithms and decision-making processes do not reinforce or perpetuate traditional gender stereotypes")
policy_sentences.append("Our AI must be designed to avoid any form of sexual bias, ensuring that outputs and decisions do not discriminate based on sexual orientation or identity")
policy_sentences.append("The system is required to respect diverse religious beliefs, ensuring that no decision or content produced discriminates against or favors any particular faith")
policy_sentences.append("All data sources and model outputs will be routinely audited to prevent racial bias, ensuring that individuals from every racial and ethnic background receive equitable treatment")
policy_sentences.append("Our AI models shall incorporate universal design principles and undergo rigorous testing to ensure that they do not disadvantage individuals with disabilities, thereby promoting accessibility and inclusion")

policy_sentences

["NATIONAL SCIENCE FOUNDATION2415 EISENHOWER AVENUEALEXANDRIA, VIRGINIA 22314NSF 19-018Dear Colleague Letter: EArly-concept Grants for ExploratoryResearch on Artificial Intelligence (AI) and Society - SupportedJointly with the Partnership on AINovember 1 5, 2018Dear Colleagues:The National Science Foundation's (NSF) Directorates for Computer and Information Scienceand Engineering (CISE) and Social, Behavioral and Economic Sciences (SBE) together withthe Partnership on AI (PAI) wish to notify the community of their interest in supporting EArly-concept Grants for Exploratory Research (EAGERs) to understand the social challengesarising from AI technology and enable scientific contributions to overcome them",
 'The last 20 years have seen rapid advances in machine learning, pattern recognition,planning, effective decision making, natural language processing, and machine vision',
 'Theseadvances have been fueled by increased data, faster computation, and improved algorithms',
 'They are yie

In [32]:
df = pd.DataFrame(policy_sentences, columns=["comment_text"])

# Tokenize each sentence with a maximum token count of 128
df["tokenized"] = df["comment_text"].apply(
    lambda text: tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        add_special_tokens=True
    )
)

In [33]:
class SentenceDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokenized = row['tokenized']
        input_ids = torch.tensor(tokenized['input_ids'])
        attention_mask = torch.tensor(tokenized['attention_mask'])
        # Some tokenizers include token_type_ids; if available, include them
        if 'token_type_ids' in tokenized:
            token_type_ids = torch.tensor(tokenized['token_type_ids'])
        else:
            token_type_ids = None

        if token_type_ids is not None:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'token_type_ids': token_type_ids
            }
        else:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }

# Create the dataset
sentence_dataset = SentenceDataset(df)

In [34]:
sentence_dataset

In [35]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,   # Not needed if only predicting
    do_eval=False
)

trainer = Trainer(
    model=model,
    args=training_args
)

In [36]:
predictions_output = trainer.predict(sentence_dataset)

In [37]:
probabilities = torch.sigmoid(torch.tensor(predictions_output.predictions))

In [38]:
bias_columns = ['gender_bias', 'sexual_bias', 'religion_bias', 'race_bias', 'disability_bias']
predictions_df = pd.DataFrame(probabilities, columns=bias_columns)

df_with_predictions = pd.concat([df.reset_index(drop=True), predictions_df], axis=1)
df_with_predictions[bias_columns] = df_with_predictions[bias_columns].mask(df_with_predictions[bias_columns] < 0.05, 0).round(3)
df_with_predictions

,comment_text,tokenized,gender_bias,sexual_bias,religion_bias,race_bias,disability_bias
0,"NATIONAL SCIENCE FOUNDATION2415 EISENHOWER AVENUEALEXANDRIA, VIRGINIA 22314NSF 19-018Dear Colleague Letter: EArly-concept Grants for ExploratoryResearch on Artificial Intelligence (AI) and Society - SupportedJointly with the Partnership on AINovember 1 5, 2018Dear Colleagues:The National Science Foundation's (NSF) Directorates for Computer and Information Scienceand Engineering (CISE) and Social, Behavioral and Economic Sciences (SBE) together withthe Partnership on AI (PAI) wish to notify the community of their interest in supporting EArly-concept Grants for Exploratory Research (EAGERs) to understand the social challengesarising from AI technology and enable scientific contributions to overcome them","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
1,"The last 20 years have seen rapid advances in machine learning, pattern recognition,planning, effective decision making, natural language processing, and machine vision","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
2,"Theseadvances have been fueled by increased data, faster computation, and improved algorithms","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
3,They are yielding increasingly diverse and large-scale applications deployed in settingssubject to unanticipated challenges with complex social effects,"[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
4,NSF has long supported fundamental research enabling AI technology,"[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
5,"With increases in thescale and diversity of deployments comes the need to better understand AI in the open world,including unforeseen circumstances and social impacts, and to craft approaches to AI thatconsider these from the start","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
6,"Vital directions include developing principles for safe, robust,and trustworthy AI (including shared responsibilities between humans and AI systems);addressing issues of bias, fairness, and transparency of algorithmic intelligence; developingdeeper understanding of human-AI interaction and user education; and developing insightsabout the influences of AI on people and society","[input_ids, attention_mask]",0.054,0.0,0.0,0.000,0.000
7,"NSF and PAI will jointly support high-risk, high-reward research at the intersection of thesocial and technical dimensions of AI","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
8,"Priority will be given to collaborative projects thatintegrate computer/computational science with the social, behavioral, and economicsciences","[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
9,Proposals may expand understanding of the influences of AI on people andsociety or contribute technical innovations that overcome the emerging social challenges,"[input_ids, attention_mask]",0.000,0.0,0.0,0.000,0.000
